#### Basado en el 7.1 (feature con lujos), cambio el encoding a label encoding.

In [32]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt
import utils.encodeador as my_encoder
import utils.featurizerV2 as ft_v2
from datetime import datetime

import re

from math import sqrt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


In [15]:
df_test = pd.read_csv("data/test.csv")

In [16]:
train = ft_v2.featurizerv2(train,"train")
test = ft_v2.featurizerv2(test,"test")

In [17]:
train.shape

(240000, 56)

In [18]:
test.shape

(60000, 55)

In [19]:
drop_cols = ['id','titulo','descripcion','direccion','fecha']

In [20]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [21]:
X.columns

Index(['tipodepropiedad', 'ciudad', 'provincia', 'antiguedad', 'habitaciones',
       'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'lat',
       'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'seguridad', 'plazas_y_derivados',
       'patio_o_jardin', 'en_avenida', 'con_referencias_acuaticas',
       'cantidad_de_beneficios', 'con_referencias_turisticas', 'es_habitable',
       'dolar', 'devaluacion_acumulada', 'year', 'mes', 'dia_del_mes',
       'dia_del_anio', 'quarter', 'dia_de_semana', 'es_finde', 'aire_libre',
       'is_DF', 'x', 'y', 'z', 'mean_metroscubiertos_tipodepropiedad',
       'mean_metrostotales_tipodepropiedad', 'mean_gimnasio_tipodepropiedad',
       'mean_usosmultiples_tipodepropiedad', 'mean_piscina_tipodepropiedad',
       'mean_escuelascercanas_tipodepropiedad',
       'mean_centroscomercialescercanos_tipodepropiedad',
       'mean_garages_tipodepropiedad', 'mean_banos_tipodepropiedad',

In [22]:
X = my_encoder.label_encoder(X)
test = my_encoder.label_encoder(test)

In [23]:
X.shape

(240000, 50)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
hyperOpt = {
    'objective' :'reg:squarederror', 
    'colsample_bytree' : 0.5, 
    'learning_rate' : 0.1,
    'max_depth' : 15,
    'alpha' : 10,
    'n_estimators' : 300,
    'eval_metric' : 'mae',
    'subsample' : 1,
    'min_child_weight' : 25,
    'gamma' : 5
}

In [26]:
xgb = XGBRegressor(**hyperOpt)

In [28]:
parameters = {
    'max_depth': [5, 10, 15, 20],
    #'n_estimators': [300, 500],saqué el 100 y 1000
    'min_child_weight': [1, 5, 10, 25],
    'learning_rate': [0.1, 0.01, 0.05],
    'subsample': [0.5,0.8,1],
    'colsample_bytree': [0.5,0.8,1],
    'gamma': [0.5, 1, 2, 5],
    'colsample_bytree': [0.5, 0.75, 1]
}


In [29]:
#gs = GridSearchCV(xgb, param_grid= parameters, scoring='neg_median_absolute_error',n_jobs=3)
randomSearchXGB = RandomizedSearchCV(xgb, parameters, scoring ='neg_mean_absolute_error', n_jobs=-1, n_iter= 15, verbose= 10)

In [33]:
datetime.now()

datetime.datetime(2019, 12, 3, 20, 44, 44, 520915)

In [34]:
%%time
#gs.fit(X_train,y_train)
randomSearchXGB.fit(X_train,y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


/home/braianvillalba/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 10.2min
/home/braianvillalba/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 31.6min
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed: 44.4min remaining: 12.7min
[Parallel(n_jobs=-1)]: Done  40 out 

CPU times: user 6min 35s, sys: 822 ms, total: 6min 35s
Wall time: 1h 1min 51s


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=XGBRegressor(alpha=10, base_score=0.5,
                                          booster='gbtree', colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.5,
                                          eval_metric='mae', gamma=5,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=15, min_child_weight=25,
                                          missing=None, n_estimators=300,
                                          n_jobs=1, nthread=N...
                                          verbosity=1),
                   iid='warn', n_iter=15, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5, 0.75, 1],
                                        'gamma

In [36]:
best_params_rs = randomSearchXGB.best_params_

In [37]:
best_params_rs

{'subsample': 1,
 'min_child_weight': 25,
 'max_depth': 20,
 'learning_rate': 0.05,
 'gamma': 0.5,
 'colsample_bytree': 0.75}

In [40]:
hyperOpt.update(best_params_rs)

In [41]:
hyperOpt

{'objective': 'reg:squarederror',
 'colsample_bytree': 0.75,
 'learning_rate': 0.05,
 'max_depth': 20,
 'alpha': 10,
 'n_estimators': 300,
 'eval_metric': 'mae',
 'subsample': 1,
 'min_child_weight': 25,
 'gamma': 0.5}

In [42]:
xgb = XGBRegressor(**hyperOpt)

In [43]:
%%time
xgb.fit(X_train, y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 7min 37s, sys: 380 ms, total: 7min 37s
Wall time: 7min 38s


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75, eval_metric='mae',
             gamma=0.5, importance_type='gain', learning_rate=0.05,
             max_delta_step=0, max_depth=20, min_child_weight=25, missing=None,
             n_estimators=300, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [44]:
pred = xgb.predict(X_test)

In [45]:
mean_absolute_error(y_test,pred) ##532394.063667334

530167.3339486573

In [46]:
predicion = xgb.predict(test)

In [48]:
resultado = pd.DataFrame(predicion, index=df_test.id, columns=['target'])
display(resultado.head())
resultado.to_csv("data/workshop-xgboost-v7_2.csv", header=True) 

,target
id,
4941,5.657916e+06
51775,1.330700e+06
115253,2.350080e+06
299321,1.480108e+06
173570,5.640928e+05
